[View in Colaboratory](https://colab.research.google.com/github/jonaskratochvil/hello-world/blob/master/Moje_konecne_funkcni_NN_mini.ipynb)

In [0]:
import numpy as np

import random 

from scipy.stats import truncnorm

In [0]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

def stable_softmax(x):
  exps = np.exp(X - np.max(X))
  return exps / np.sum(exps)     

In [0]:
def truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [0]:
class NeuralNetwork:
  
  def __init__(self, n_in_nodes, n_hidden_nodes, n_out_nodes, learning_rate):
    
    self.n_in_nodes = n_in_nodes
    self.n_hidden_nodes = n_hidden_nodes
    self.n_out_nodes = n_out_nodes
    self.learning_rate = learning_rate
    self.weights_in = np.random.randn(n_hidden_nodes, n_in_nodes)
    self.weights_out = np.random.randn(n_out_nodes, n_hidden_nodes)
    self.bias_in = np.random.randn(n_hidden_nodes,1)
    self.bias_out = np.random.randn(n_out_nodes,1)
  
  def train(self, input_data, input_labels):
    
    # Transponovani dat z 4x3 na 3x4 -> weights jsou 4x3, 1x4
    
    input_data = np.array(input_data, ndmin = 2).T
    input_labels = np.array(input_labels, ndmin = 2).T
    
    # Forward propagate 4x3 x 3x4 -> 4x4 
    
    l0 = sigmoid(np.dot(self.weights_in, input_data)+self.bias_in)
    
    # 1x4 x 4x4 -> 1x4
    
    l1 = sigmoid(np.dot(self.weights_out, l0)+self.bias_out)
    
    # self.output_error_L2 = 1/2*(input_data - l1) **2
    # d1 = l1 - input_data * (l1)(1-l1) -> tady je ten term derivace sigmoidu ktery spomaluje learning
    
    self.output_error_CE = -(input_labels * np.log(l1) + (1-input_labels) * np.log(1-l1))
    
    # dal jsem derivaci square loss kdyz napr 0.8 - 0 -> zaporny learning rate da to znamenko tak jak chci
    
    # kdyz 0.1 - 1 tak zase ten learning rate to hodi tam kam chci
    
    # Backpropagate -> deltas in first and zero layer from Nielsen formula
    
    # i d1 nyni derivace CE funkce kterou se vymaze ta derivace sigmoidu -> opravdu je videt rozdil!!!
    
    d1 = -input_labels*(1-l1) + (1-input_labels)*l1
    d0 = np.dot(self.weights_out.T,d1)*(l0*(1-l0))
    
    # Weight update change from Nielsen formula to np.dot(delta1, output0.T)
    
    self.dw_out = np.dot(d1,l0.T)
    self.dw_in = np.dot(d0,input_data.T)
    
    self.db_out = d1
    self.db_in = d0
    
  def SGD(self, X, y, epochs, batch_size):
    
    batch_w_in_update = np.zeros((self.n_hidden_nodes, self.n_in_nodes), dtype=int)
    batch_w_out_update = np.zeros((self.n_out_nodes, self.n_hidden_nodes), dtype=int)
    batch_b_in_update = np.zeros((self.n_hidden_nodes,1), dtype=int)
    batch_b_out_update = np.zeros((self.n_out_nodes,1), dtype=int)
    
    for epoch in range(epochs):
      
      # shuffle randomly data before start of each epoch
      
      data_shuffle = [i for i in range(X.shape[0])]
      data_shuffle = random.sample(data_shuffle, len(data_shuffle))

      X = np.array([X[i] for i in data_shuffle])
      y = np.array([y[i] for i in data_shuffle])
      
      for minibatch in range(0,X.shape[0],batch_size):
        
        for example in range(minibatch,minibatch+batch_size):
          
          # Append weight and biases gradients
          
          self.train(X[example],y[example])
          batch_w_in_update = batch_w_in_update + self.dw_in
          batch_w_out_update = batch_w_out_update + self.dw_out
          batch_b_in_update = batch_b_in_update + self.db_in
          batch_b_out_update = batch_b_out_update + self.db_out
        
        # Update current weights to new ones with minibatch update
        
        self.weights_in = self.weights_in - self.learning_rate/batch_size * batch_w_in_update 
        self.weights_out = self.weights_out - self.learning_rate/batch_size * batch_w_out_update
        
        self.bias_in = self.bias_in - self.learning_rate/batch_size * batch_b_in_update 
        self.bias_out = self.bias_out - self.learning_rate/batch_size * batch_b_out_update
        
        batch_w_in_update = np.zeros((self.n_hidden_nodes, self.n_in_nodes), dtype=int)
        batch_w_out_update = np.zeros((self.n_out_nodes, self.n_hidden_nodes), dtype=int)
        batch_b_in_update = np.zeros((self.n_hidden_nodes,1), dtype=int)
        batch_b_out_update = np.zeros((self.n_out_nodes,1), dtype=int)
        
      if (epoch % 20000) == 0:

        print ("epoch:" + str(epoch) + " error: " + str(np.mean(np.abs(self.output_error_CE))))
    
  def run(self, test_data):
     
    test_data = np.array(test_data, ndmin = 2).T
    l0 = sigmoid(np.dot(self.weights_in, test_data) + self.bias_in)
    l1 = sigmoid(np.dot(self.weights_out, l0) + self.bias_out)

    return l1
    
    
    

In [0]:
net = NeuralNetwork(3, 4, 1, 0.001)

In [0]:
X = np.array([[0,0,1],
             [0,1,1],
             [1,0,1],
             [1,1,1],])

y = np.array([[0,0,1,1]]).T


In [363]:
net.SGD(X,y,200000,2)

epoch:0 error: 0.5447801692479085
epoch:20000 error: 0.2364271461453347
epoch:40000 error: 0.05006288139600575
epoch:60000 error: 0.03709872342101401
epoch:80000 error: 0.017374361830095466
epoch:100000 error: 0.018546471176214486
epoch:120000 error: 0.010387390881346955
epoch:140000 error: 0.00900982850236944
epoch:160000 error: 0.009923341906941593
epoch:180000 error: 0.008683635833432296


In [353]:
print(net.weights_out)

print(net.weights_in)

[[-2.30590105  1.66158096 -3.42337182 -1.60904191]]
[[-2.44224578 -0.07899151  0.9947154 ]
 [ 1.07269503 -0.1453175   0.39861019]
 [-3.32550781  0.48899724 -1.21814016]
 [-1.89712127 -1.4544356  -0.51458949]]


In [365]:
point = np.array([[0,0,1]])

net.run(point)

array([[0.00760154]])

In [0]:
for i in range(100000):
  net.train(X,y)
  if (i % 10000) == 0:

      print ("iteration:" + str(i) + " accuracy: " + str(1 - np.mean(np.abs(net.output_error))))

In [148]:
v = ["bear", "lion", "dog"]

z = []

for i in v:
  z.append(i)
  

ind_list = [i for i in range(len(z))]

ind_list = random.sample(ind_list, len(ind_list))

z = [z[i] for i in ind_list]

print(z)




['lion', 'dog', 'bear']


In [0]:
cross_entropy_loss = -(1.0/intput_data.T.shape[0]) * np.sum(input_labels*np.log(l1) + (1-input_labels)*np.log(1-l1))


In [0]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST.data/", one_hot = True)

In [5]:
print(mnist.train.labels.shape)

(55000, 10)


In [0]:
net_mnist = NeuralNetwork(784, 10 , 10, 0.1)

In [0]:
for i in range(1000):
  net_mnist.train(mnist.train.images,mnist.train.labels)
  
  if i % 100 == 0:
    
    print ("iteration:" + str(i) + " accuracy: " + str(1 - np.mean(np.abs(net_mnist.output_error))))

In [10]:
print(mnist.test.labels[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [0]:
net_mnist.run(mnist.test.images[0])